In [ ]:
def volumeEstimate(a,b,c,d,n,f,points):
    #very useful for estimating volume once you've picked points
    """Given a list of points, a region, and a function, estimates the volume between the region and the surface"""
    points = points.values.tolist()
    list1 = []
    dx = (b-a)/n
    dy = (d-c)/n
    for i in range(len(points)):
        list1.append((dx + a) * (dy + c) * f(points[i][0],points[i][1]))
    return sum(list1)

In [1]:
def MeshXVolumeEstimator(f,xVals,points,a,b,c,d):
    """Parameters
    f: A function
    xVals: a list of x values corresponding to vertical lines
    points: a list of the points on the region
    a,b,c,d: coordinates of the region [a,b] x [c,d]
    
    Returns an estimate of the volume between the surface and the region based on these inputs
    Designed to work with a mesh analysis technique that recursively divides the x-axis   """
    height = d - c
    df = pd.DataFrame()
    volumes = []
    oldX = [] #used for contour plot
    oldY= [] #used for contour plot
    xs = xVals[:] #clone list so that function can be reused without inserting into xVals every time
    xs.insert(0,0)
    xs.insert(len(xs),b)
    xs = np.array(xs) #NUMPIZE xs
    dxs = xs[1:] - xs[:-1] #intervals between x-values
    for i in range(1,len(xs)):
        point = filterbox_x(points,xs[i-1],xs[i],3,3,0,1) #get all points from each region - no need to account for case when
        #there isn't a point because there always will be since we did mesh analysis
        centerx = (xs[i] - xs[i-1])/2 + xs[i-1] #center x value of each region
        centery = (height)/2
        oldX.append(centerx)
        oldY.append(centery)
        closest = closestDistance(centerx,centery,point)
        estimatePoint = point.iloc[closest] #find point closest to the center of each subarea
        df = df.append(estimatePoint, ignore_index = True)
    dfOld = pd.DataFrame({'Old X': oldX, 'Old Y': oldY})
    for i in range(len(dxs)):
        volumes.append(dxs[i] * height * f(df.iloc[i]['x'],df.iloc[i]['y'])) #volume estimate of each subarea
    print(sum(volumes))
    return df.join(dfOld) #return table

In [ ]:
def MeshYVolumeEstimator(f,yVals,points,a,b,c,d):
    """Parameters
    f: A function
    yVals: a list of y values corresponding to horizontal lines
    points: a list of the points on the region
    a,b,c,d: coordinates of the region [a,b] x [c,d]

    Returns an estimate of the volume between the surface and the region based on these inputs
    Designed to work with a mesh analysis technique that recursively divides the y-axis   """
    height = d - c
    width = b - a
    df = pd.DataFrame()
    volumes = []
    oldX = [] #used for contour plot
    oldY= [] #used for contour plot
    ys = yVals[:] #clone list so that function can be reused without inserting into yVals every time
    ys.insert(0,0)
    ys.insert(len(ys),d)
    ys = np.array(ys) #NUMPIZE ys ;)
    dys = ys[1:] - ys[:-1] #intervals between y-values
    for i in range(1,len(ys)):
        point = filterbox_y(points,3,3,ys[i-1],ys[i],0,1) #get all points from each region - no need to account for case when
        #there isn't a point because there always will be since we did mesh analysis
        centerx = (width)/2 #center x value of each region
        centery = (ys[i] - ys[i-1])/2 + ys[i-1] #center y value of each region
        oldX.append(centerx)
        oldY.append(centery)
        closest = closestDistance(centerx,centery,point)
        estimatePoint = point.iloc[closest] #find point closest to the center of each subarea
        df = df.append(estimatePoint, ignore_index = True)
    dfOld = pd.DataFrame({'Old X': oldX, 'Old Y': oldY})
    for i in range(len(dys)):
        volumes.append(dys[i] * width * f(df.iloc[i]['x'],df.iloc[i]['y'])) #volume estimate of each subarea
    print(sum(volumes))
    return df.join(dfOld) #return table